In [ ]:
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import Credentials, APIClient
from dotenv import load_dotenv
import os
load_dotenv()

WML_URL=os.getenv("WML_URL","")
CLOUD_API_KEY=os.getenv("CLOUD_API_KEY","")
LLM_MODEL=os.getenv("LLM_MODEL","")
DEPLOYMENT_SPACE_ID=os.getenv("DEPLOYMENT_SPACE_ID","")
creds = Credentials(
                url=WML_URL,
                api_key=CLOUD_API_KEY
)
param = {
        "decoding_method": "greedy"
}


def get_llm()->ChatWatsonx:
    client = APIClient(creds)
    client.set.default_space(DEPLOYMENT_SPACE_ID)
    model = ChatWatsonx(
            model_id=LLM_MODEL,
            watsonx_client=client,
            params=param
        )
    return model
llm=get_llm()

In [ ]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import Literal

In [ ]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

In [ ]:
get_weather.invoke(input={"location": "munich"})

In [ ]:
tools=[get_weather]
model=llm.bind_tools(tools)

In [ ]:
model.invoke("hello")

In [ ]:
model.invoke("How is the weather in munich?")

In [ ]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [ ]:
workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

In [ ]:
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
from langchain_core.messages import HumanMessage

messages1 = [HumanMessage(content="Hello, how are you?")]
messages2 = [HumanMessage(content="How is the weather in munich?")]

In [ ]:
graph.invoke({"messages": messages1})

In [ ]:
graph.invoke({"messages": messages2})

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city than?")
        ]
    }
)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()


In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
graph.invoke(
    {"messages": [HumanMessage(content="How is the weather in munich?")]},
    config={"configurable": {"thread_id": 1}},
)

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city than?")
        ]
    },
    config={"configurable": {"thread_id": 1}},
)